##  Vorbeitung der Daten für eine erste Datenanalyse (Datenreinigung)

In diesem Notebook sollen die geparsten Daten aus dem Notenook [MedReg_Parsing_04](MedReg_Parsing_04.ipynb) einer ersten Analyse unterzogen werden. 
Im Hinblick auf den Beitrag in der Sendung vom 17.2.2020 sollen die im [csv-File]( http://localhost:8888/edit/Desktop/CAS_DJ/MedReg/MedReg%20Scraping%202020/medreg_2020_psych_20200121.csv) abgelegten «dreckigen» Daten dabei möglichst folgenden, (hoffentlich) einfacheren Analysen unterzogen werden:

1. Stand der Facharztabschlüsse und Nationalität per Ende 2019 (auf Basis von Land Medizinsudium und der direkten Anerkennungen ausländischer Facharzttiteln
2. Aussagen über die Nationalität bzw. Studienort der anerkannten Weiterbildungstitel
4. Anteil aller Psychiater mit ausländischer Herkunft per Ende 2019


In weiteren Analysen sollen später auch weitere Fragen untersucht werden, wie:

5. Entwicklung Gesamtanteil über die Jahre, bis Ende 2019
6. Entwicklung Anteil der Anerkennung (sprich: Neuzulassung) per Jahresende, über die Jahre, bis Ende 2019
7. regionale Aussagen über die Tätigkeitsorte (Adresse der Bewilligung) wie zum Beispiel:
  - Vergleich zur Kantonsbevölkerung? (aussagekräftig?)
  - Städtische Gebiete, ländliche Gebiete? (aufwändig)
  - Grossstädte vs. landliche Zentren?

In [4]:
import pandas as pd
import re
import numpy as np
import datetime

In [98]:
df = pd.read_csv('medreg_2020_psych_20200121.csv')

In [99]:
df.head(2)

,Unnamed: 0,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e)
0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung \n (28.03.2007)...,<tbody>\n<tr>\n<td>Ärztin/Arzt</td>\n<td>2006<...,Nationalität: Österreich (AT),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Aarg..."
1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,<tbody>\n<tr>\n<td>Ärztin/Arzt</td>\n<td>1982<...,Nationalität: Schweiz (CH),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Base..."


In [100]:
df['Land Diplom'].value_counts()

Schweiz                      2837
Deutschland                  1365
Frankreich                    318
Italien                       214
Österreich                    180
Spanien                        73
Rumänien                       63
Griechenland                   58
vorm. Jugoslawien              42
Belgien                        41
Bulgarien                      22
Polen                          21
Algerien                       19
Ungarn                         17
vorm. Sowjetunion              14
Russische Föderation           14
Niederlande                    12
Portugal                       10
Slowakei                        8
Serbien Republik                8
Grossbritannien                 8
Peru                            7
Argentinien                     6
Brasilien                       6
Kroatien                        5
Tschechische Republik           5
Ukraine                         4
Ägypten                         4
Slowenien                       4
Bosnien und He

### Datensatz bereinigen 
Vor den ersten Auswertungen müssen wir die Daten noch weiter bereinigen bzw. «säubern». 
Die Herausforderung: bei den Weiterbildungstiteln die richtigen Datenpaare *Jahr* und *Psychiatrie und Psychotherapie* zu extrahieren. 

Wir müssen dabei folgende Textstelle ausfindig machen:

    <td>Psychiatrie und Psychotherapie</td>
    <td>2013</td>
    <td>Schweiz</td>
    <td>Eidgenössischer Weiterbildungstitel</td>
    </tr>
    
Wir versuchen das mit Regex.

Oder zuerst doch mit BS?
[Stackoverflow-Tipp](https://stackoverflow.com/questions/44703945/pandas-trouble-stripping-html-tags-from-dataframe-column)


In [101]:
import bs4

df['Ausbildung'] = df['Ausbildung'].apply(lambda x: bs4.BeautifulSoup(x, 'lxml').get_text())

In [103]:
df.head(2)

,Unnamed: 0,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e)
0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung \n (28.03.2007)...,\n\nÄrztin/Arzt\n2006\nÖsterreich\nAnerkennung...,Nationalität: Österreich (AT),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Aarg..."
1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,\n\nÄrztin/Arzt\n1982\nSchweiz\nEidgenössische...,Nationalität: Schweiz (CH),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Base..."


In [104]:
df.to_csv(r'/Users/master/Desktop/CAS_DJ/MedReg/MedReg Scraping 2020/medreg_2020_psych_20200124.csv')

Wir haben nun folgende Muster, die wir finden müssen:
    
    Psychiatrie und Psychotherapie
    2008
    Deutschland
    Anerkennung 
                        (31.01.2011)
                        
                        
 oder 
 
    Psychiatrie und Psychotherapie
    2013
    Schweiz
    Eidgenössischer Weiterbildungstitel

Und jetzt mit Regex weiter.

In [105]:
df['Ausbildung'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)

In [106]:
df.head(3)

,Unnamed: 0,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e)
0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung \n (28.03.2007)...,Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Nationalität: Österreich (AT),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Aarg..."
1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Nationalität: Schweiz (CH),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Base..."
2,0,medreg_68248.html,"Klein, Katrin Stephanie",w,"Mühlemattstrasse 62,3007 Bern,BE",2000,Deutschland,Anerkennung \n (26.01.2011)...,Ärztin/Arzt 2000 Deutschland Anerkennung (26....,Nationalität: Deutschland (DE),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Bern..."


In [107]:
df['Anerkennung'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)

In [108]:
df['Anerkennung']

0       Anerkennung (28.03.2007) 
1          Eidgenössisches Diplom
2       Anerkennung (26.01.2011) 
3       Anerkennung (23.07.2002) 
4          Eidgenössisches Diplom
                  ...            
5432       Eidgenössisches Diplom
5433       Eidgenössisches Diplom
5434    Anerkennung (15.07.2002) 
5435    Anerkennung (22.06.2016) 
5436       Eidgenössisches Diplom
Name: Anerkennung, Length: 5437, dtype: object

In [109]:
df['Ausbildung']

0        Ärztin/Arzt 2006 Österreich Anerkennung (28.0...
1        Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...
2        Ärztin/Arzt 2000 Deutschland Anerkennung (26....
3        Ärztin/Arzt 1994 Deutschland Anerkennung (23....
4        Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...
                              ...                        
5432     Ärztin/Arzt 2004 Schweiz Eidgenössisches Dipl...
5433     Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...
5434     Ärztin/Arzt 1992 Deutschland Anerkennung (15....
5435     Ärztin/Arzt 2008 Deutschland Anerkennung (22....
5436     Ärztin/Arzt 2006 Schweiz Eidgenössisches Dipl...
Name: Ausbildung, Length: 5437, dtype: object

In [110]:
df['PP_Jahr'] = df['Ausbildung'].str.extract(r"Psychiatrie und Psychotherapie (\d{4})")

In [111]:
df['PP_Jahr']

0       2013
1       1989
2       2008
3       2002
4       1990
        ... 
5432    2013
5433    1992
5434    2002
5435    2019
5436    2018
Name: PP_Jahr, Length: 5437, dtype: object

In [112]:
df.head(3)

,Unnamed: 0,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr
0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Nationalität: Österreich (AT),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Aarg...",2013
1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Nationalität: Schweiz (CH),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Base...",1989
2,0,medreg_68248.html,"Klein, Katrin Stephanie",w,"Mühlemattstrasse 62,3007 Bern,BE",2000,Deutschland,Anerkennung (26.01.2011),Ärztin/Arzt 2000 Deutschland Anerkennung (26....,Nationalität: Deutschland (DE),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Bern...",2008


In [113]:
df['PP_Land'] = df['Ausbildung'].str.extract(r"Psychiatrie und Psychotherapie\s\d{4}\s(\w*)")

In [114]:
df['PP_Land']

0           Schweiz
1           Schweiz
2       Deutschland
3           Schweiz
4           Schweiz
           ...     
5432        Schweiz
5433        Schweiz
5434        Schweiz
5435    Deutschland
5436        Schweiz
Name: PP_Land, Length: 5437, dtype: object

In [115]:
df['PP_Land'].value_counts()

Schweiz            3905
Deutschland         869
Frankreich          315
Italien             122
Österreich           50
Belgien              30
Spanien              29
Rumänien             26
Griechenland         14
Ungarn                9
Bulgarien             9
Grossbritannien       9
Schweden              8
Niederlande           8
Portugal              7
Kroatien              2
Norwegen              2
Finnland              2
Tschechische          2
Slowenien             1
Lettland              1
Vereinigte            1
Serbien               1
ehem                  1
Polen                 1
Slowakei              1
Estland               1
Name: PP_Land, dtype: int64

In [116]:
df.head(2)

,Unnamed: 0,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land
0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Nationalität: Österreich (AT),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Aarg...",2013,Schweiz
1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Nationalität: Schweiz (CH),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Base...",1989,Schweiz


In [117]:
df['PP_Anerkennung'] = df['Ausbildung'].str.extract(r"Psychiatrie und Psychotherapie\s\d{4}\s.*Anerkennung\s\((\d{2}.\d{2}.\d{4})\)")

In [118]:
df.head(3)

,Unnamed: 0,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land,PP_Anerkennung
0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Nationalität: Österreich (AT),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Aarg...",2013,Schweiz,NaN
1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Nationalität: Schweiz (CH),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Base...",1989,Schweiz,NaN
2,0,medreg_68248.html,"Klein, Katrin Stephanie",w,"Mühlemattstrasse 62,3007 Bern,BE",2000,Deutschland,Anerkennung (26.01.2011),Ärztin/Arzt 2000 Deutschland Anerkennung (26....,Nationalität: Deutschland (DE),Sprachkenntnisse: Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Bern...",2008,Deutschland,09.11.2016


Endlich, nach vielen Versuchen mit Regex (ca. 2-3 Stunden) geschafft, sämtliche relevanten Weiterbildungsdaten in neue Spalten zu überführen. 
Weiter mit dem Reinigen der restlichen Dateien:

In [119]:
df['Nationalität'] = df.Nationalität.str.replace('Nationalität:' , '')

In [120]:
df['Sprache'] = df.Sprache.str.replace('Sprachkenntnisse:' , '')

In [121]:
df.drop(['Unnamed: 0'], axis=1)

,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land,PP_Anerkennung
0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Österreich (AT),Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Aarg...",2013,Schweiz,NaN
1,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Base...",1989,Schweiz,NaN
2,medreg_68248.html,"Klein, Katrin Stephanie",w,"Mühlemattstrasse 62,3007 Bern,BE",2000,Deutschland,Anerkennung (26.01.2011),Ärztin/Arzt 2000 Deutschland Anerkennung (26....,Deutschland (DE),Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Bern...",2008,Deutschland,09.11.2016
3,medreg_15778.html,"Leinfelder, Holger Ralf",m,"Austr. 79,4051 Basel,BS",1994,Deutschland,Anerkennung (23.07.2002),Ärztin/Arzt 1994 Deutschland Anerkennung (23....,Deutschland (DE),Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Base...",2002,Schweiz,NaN
4,medreg_34926.html,"Jobin, Peter",m,"Standweg 20,2540 Grenchen,SO",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Solo...",1990,Schweiz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5432,medreg_17947.html,"Marti, Aurélie Sabine Daphné",w,"Rue des Lattes 59,1217 Meyrin,GE",2004,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 2004 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Genf...",2013,Schweiz,NaN
5433,medreg_55667.html,"Nick, Beat",m,"Weissensteinstrasse ,4503 Solothurn,SO",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Solo...",1992,Schweiz,NaN
5434,medreg_15583.html,"Jung, Gerhard Franz",m,"Leutholdstr. 4,8620 Wetzikon,ZH",1992,Deutschland,Anerkennung (15.07.2002),Ärztin/Arzt 1992 Deutschland Anerkennung (15....,Deutschland (DE),Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Züri...",2002,Schweiz,NaN
5435,medreg_93501.html,"Blasius, Elias John Achim",m,Kein Eintrag,2008,Deutschland,Anerkennung (22.06.2016),Ärztin/Arzt 2008 Deutschland Anerkennung (22....,Deutschland (DE),Deutsch,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2019,Deutschland,24.04.2019


In [122]:
df['BAB Status'].value_counts()

Erteilt                                                                           4428
Direktabgabe von Arzneimitteln gemäss kant. Bestimmungen (Selbstdispensation)      986
90-Tage Dienstleister                                                               12
Keine Bewilligung                                                                   11
Name: BAB Status, dtype: int64

In [123]:
df.to_csv(r'/Users/master/Desktop/CAS_DJ/MedReg/MedReg Scraping 2020/medreg_2020_psych_20200124_2.csv')

In [145]:
df = pd.read_csv(r'/Users/master/Desktop/CAS_DJ/MedReg/MedReg Scraping 2020/medreg_2020_psych_20200124_2.csv')

In [146]:
df['Anerkennung Arztdiplom'] = df['Anerkennung'].str.extract(r"Anerkennung\s\((\d{2}.\d{2}.\d{4})\)")

In [147]:
df.head(2)

,Unnamed: 0,Unnamed: 0.1,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land,PP_Anerkennung,Anerkennung Arztdiplom
0,0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Österreich (AT),Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Aarg...",2013.0,Schweiz,NaN,28.03.2007
1,1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"<ol style=""list-style-type: circle"">\n<li>Base...",1989.0,Schweiz,NaN,NaN


Auch die Spalte *BAB Kanton(e)* soll noch etwas gesäubert werden, z.B. damit:

In [148]:
df['BAB Kanton(e)'] = df['BAB Kanton(e)'].apply(lambda x: bs4.BeautifulSoup(x, 'lxml').get_text())

In [149]:
df['BAB Kanton(e)'].str.replace('\n', '')

0       Aargau                (2018), aktiv, MedBG, pr...
1       Basel-Land                (1990), aktiv, MedBG...
2       Bern                (2011), aktiv, MedBG, priv...
3       Basel-Stadt                (2007), aktiv, Unbe...
4       Solothurn                (1990), aktiv, MedBG,...
                              ...                        
5432    Genf                (2013), aktiv, MedBG, priv...
5433    Solothurn                (1998), aktiv, Kt. Re...
5434    Zürich                (2002), aktiv, MedBG, pr...
5435                                         Kein Eintrag
5436    Waadt                (2019), aktiv, MedBG, pri...
Name: BAB Kanton(e), Length: 5437, dtype: object

In [150]:
df.head(2)

,Unnamed: 0,Unnamed: 0.1,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land,PP_Anerkennung,Anerkennung Arztdiplom
0,0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Österreich (AT),Deutsch,Erteilt,"\nAargau\n (2018), aktiv, MedBG...",2013.0,Schweiz,NaN,28.03.2007
1,1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"\nBasel-Land\n (1990), aktiv, M...",1989.0,Schweiz,NaN,NaN


In [151]:
df['BAB Kanton(e)'].replace(r'\n', '', regex=True, inplace=True)

In [152]:
df.head(2)

,Unnamed: 0,Unnamed: 0.1,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land,PP_Anerkennung,Anerkennung Arztdiplom
0,0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Österreich (AT),Deutsch,Erteilt,"Aargau (2018), aktiv, MedBG, pr...",2013.0,Schweiz,NaN,28.03.2007
1,1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Basel-Land (1990), aktiv, MedBG...",1989.0,Schweiz,NaN,NaN


In [153]:
df['Ort'] = df['Adresse'].str.extract(r"(\d{4} .*),")

In [154]:
df['Doc_ID'] = df['Datei'].str.extract(r'medreg_(\d*)')

In [155]:
df['Doc_ID'] = pd.to_numeric(df['Doc_ID'])

In [157]:
df.dtypes

Unnamed: 0                  int64
Unnamed: 0.1                int64
Datei                      object
Name                       object
Geschlecht                 object
Adresse                    object
Jahr Diplom                 int64
Land Diplom                object
Anerkennung                object
Ausbildung                 object
Nationalität               object
Sprache                    object
BAB Status                 object
BAB Kanton(e)              object
PP_Jahr                   float64
PP_Land                    object
PP_Anerkennung             object
Anerkennung Arztdiplom     object
Ort                        object
Doc_ID                      int64
dtype: object

In [160]:
df['Doc_ID'].sort_values()

4776       552
1138       566
4434       590
2805       611
4553       617
         ...  
3917    113646
2822    113811
836     113880
1564    113887
724     113984
Name: Doc_ID, Length: 5437, dtype: int64

In [161]:
df.head(5)

,Unnamed: 0,Unnamed: 0.1,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land,PP_Anerkennung,Anerkennung Arztdiplom,Ort,Doc_ID
0,0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Österreich (AT),Deutsch,Erteilt,"Aargau (2018), aktiv, MedBG, pr...",2013.0,Schweiz,NaN,28.03.2007,5620 Bremgarten AG,27259
1,1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Basel-Land (1990), aktiv, MedBG...",1989.0,Schweiz,NaN,NaN,4132 Muttenz,55166
2,2,0,medreg_68248.html,"Klein, Katrin Stephanie",w,"Mühlemattstrasse 62,3007 Bern,BE",2000,Deutschland,Anerkennung (26.01.2011),Ärztin/Arzt 2000 Deutschland Anerkennung (26....,Deutschland (DE),Deutsch,Erteilt,"Bern (2011), aktiv, MedBG, priv...",2008.0,Deutschland,09.11.2016,26.01.2011,3007 Bern,68248
3,3,0,medreg_15778.html,"Leinfelder, Holger Ralf",m,"Austr. 79,4051 Basel,BS",1994,Deutschland,Anerkennung (23.07.2002),Ärztin/Arzt 1994 Deutschland Anerkennung (23....,Deutschland (DE),Deutsch,Erteilt,"Basel-Stadt (2007), aktiv, Unbe...",2002.0,Schweiz,NaN,23.07.2002,4051 Basel,15778
4,4,0,medreg_34926.html,"Jobin, Peter",m,"Standweg 20,2540 Grenchen,SO",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Solothurn (1990), aktiv, MedBG,...",1990.0,Schweiz,NaN,NaN,2540 Grenchen,34926


In [164]:
df['Sprache'].value_counts()

 Deutsch                               3697
 Französisch                           1442
 Italienisch                            218
                                         42
 Deutsch, Italienisch                    20
 Deutsch, Französisch                     8
 Französisch, Italienisch                 7
 Deutsch, Französisch, Italienisch        1
 Deutsch, Niederländisch                  1
 Englisch, Französisch, Italienisch       1
Name: Sprache, dtype: int64

Wir brauchen für die Auswertung nur das Jahr der Anerkennung des Weiterbildungstitels, sowie des Arztdiploms. Deshalb bereinigen wir noch die zwei entsprechenden Spalten. 

In [165]:
df['PP_Anerkennung'] = df['PP_Anerkennung'].str.extract(r"(\d{4})")

In [166]:
df.head(3)

,Unnamed: 0,Unnamed: 0.1,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land,PP_Anerkennung,Anerkennung Arztdiplom,Ort,Doc_ID
0,0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Österreich (AT),Deutsch,Erteilt,"Aargau (2018), aktiv, MedBG, pr...",2013.0,Schweiz,NaN,28.03.2007,5620 Bremgarten AG,27259
1,1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Basel-Land (1990), aktiv, MedBG...",1989.0,Schweiz,NaN,NaN,4132 Muttenz,55166
2,2,0,medreg_68248.html,"Klein, Katrin Stephanie",w,"Mühlemattstrasse 62,3007 Bern,BE",2000,Deutschland,Anerkennung (26.01.2011),Ärztin/Arzt 2000 Deutschland Anerkennung (26....,Deutschland (DE),Deutsch,Erteilt,"Bern (2011), aktiv, MedBG, priv...",2008.0,Deutschland,2016,26.01.2011,3007 Bern,68248


In [167]:
df['Anerkennung Arztdiplom'] = df['Anerkennung Arztdiplom'].str.extract(r"(\d{4})")

In [168]:
df.head(5)

,Unnamed: 0,Unnamed: 0.1,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land,PP_Anerkennung,Anerkennung Arztdiplom,Ort,Doc_ID
0,0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Österreich (AT),Deutsch,Erteilt,"Aargau (2018), aktiv, MedBG, pr...",2013.0,Schweiz,NaN,2007,5620 Bremgarten AG,27259
1,1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Basel-Land (1990), aktiv, MedBG...",1989.0,Schweiz,NaN,NaN,4132 Muttenz,55166
2,2,0,medreg_68248.html,"Klein, Katrin Stephanie",w,"Mühlemattstrasse 62,3007 Bern,BE",2000,Deutschland,Anerkennung (26.01.2011),Ärztin/Arzt 2000 Deutschland Anerkennung (26....,Deutschland (DE),Deutsch,Erteilt,"Bern (2011), aktiv, MedBG, priv...",2008.0,Deutschland,2016,2011,3007 Bern,68248
3,3,0,medreg_15778.html,"Leinfelder, Holger Ralf",m,"Austr. 79,4051 Basel,BS",1994,Deutschland,Anerkennung (23.07.2002),Ärztin/Arzt 1994 Deutschland Anerkennung (23....,Deutschland (DE),Deutsch,Erteilt,"Basel-Stadt (2007), aktiv, Unbe...",2002.0,Schweiz,NaN,2002,4051 Basel,15778
4,4,0,medreg_34926.html,"Jobin, Peter",m,"Standweg 20,2540 Grenchen,SO",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Solothurn (1990), aktiv, MedBG,...",1990.0,Schweiz,NaN,NaN,2540 Grenchen,34926


In [169]:
df.dtypes

Unnamed: 0                  int64
Unnamed: 0.1                int64
Datei                      object
Name                       object
Geschlecht                 object
Adresse                    object
Jahr Diplom                 int64
Land Diplom                object
Anerkennung                object
Ausbildung                 object
Nationalität               object
Sprache                    object
BAB Status                 object
BAB Kanton(e)              object
PP_Jahr                   float64
PP_Land                    object
PP_Anerkennung             object
Anerkennung Arztdiplom     object
Ort                        object
Doc_ID                      int64
dtype: object

Nicht so easy ist es mit *PP_Jahr*, von float nach int. [Link](https://stackoverflow.com/questions/43956335/convert-float64-column-to-int64-in-pandas)

In [174]:
df['PP_Jahr'] = df['PP_Jahr'].fillna(0).astype(np.int64)

In [175]:
df.dtypes

Unnamed: 0                 int64
Unnamed: 0.1               int64
Datei                     object
Name                      object
Geschlecht                object
Adresse                   object
Jahr Diplom                int64
Land Diplom               object
Anerkennung               object
Ausbildung                object
Nationalität              object
Sprache                   object
BAB Status                object
BAB Kanton(e)             object
PP_Jahr                    int64
PP_Land                   object
PP_Anerkennung            object
Anerkennung Arztdiplom    object
Ort                       object
Doc_ID                     int64
dtype: object

In [179]:
df.rename(columns = {'Anerkennung Arztdiplom':'Arzt_Anerkennung_Jahr', 'Jahr Diplom':'Arzt_Jahr', 'Land Diplom': 'Arzt_Land', 'BAB Kanton(e)':'BAB'})

,Unnamed: 0,Unnamed: 0.1,Datei,Name,Geschlecht,Adresse,Arzt_Jahr,Arzt_Land,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB,PP_Jahr,PP_Land,PP_Anerkennung,Arzt_Anerkennung_Jahr,Ort,Doc_ID
0,0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Österreich (AT),Deutsch,Erteilt,"Aargau (2018), aktiv, MedBG, pr...",2013,Schweiz,NaN,2007,5620 Bremgarten AG,27259
1,1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Basel-Land (1990), aktiv, MedBG...",1989,Schweiz,NaN,NaN,4132 Muttenz,55166
2,2,0,medreg_68248.html,"Klein, Katrin Stephanie",w,"Mühlemattstrasse 62,3007 Bern,BE",2000,Deutschland,Anerkennung (26.01.2011),Ärztin/Arzt 2000 Deutschland Anerkennung (26....,Deutschland (DE),Deutsch,Erteilt,"Bern (2011), aktiv, MedBG, priv...",2008,Deutschland,2016,2011,3007 Bern,68248
3,3,0,medreg_15778.html,"Leinfelder, Holger Ralf",m,"Austr. 79,4051 Basel,BS",1994,Deutschland,Anerkennung (23.07.2002),Ärztin/Arzt 1994 Deutschland Anerkennung (23....,Deutschland (DE),Deutsch,Erteilt,"Basel-Stadt (2007), aktiv, Unbe...",2002,Schweiz,NaN,2002,4051 Basel,15778
4,4,0,medreg_34926.html,"Jobin, Peter",m,"Standweg 20,2540 Grenchen,SO",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Solothurn (1990), aktiv, MedBG,...",1990,Schweiz,NaN,NaN,2540 Grenchen,34926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5432,5432,0,medreg_17947.html,"Marti, Aurélie Sabine Daphné",w,"Rue des Lattes 59,1217 Meyrin,GE",2004,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 2004 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"Genf (2013), aktiv, MedBG, priv...",2013,Schweiz,NaN,NaN,1217 Meyrin,17947
5433,5433,0,medreg_55667.html,"Nick, Beat",m,"Weissensteinstrasse ,4503 Solothurn,SO",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Solothurn (1998), aktiv, Kt. Re...",1992,Schweiz,NaN,NaN,4503 Solothurn,55667
5434,5434,0,medreg_15583.html,"Jung, Gerhard Franz",m,"Leutholdstr. 4,8620 Wetzikon,ZH",1992,Deutschland,Anerkennung (15.07.2002),Ärztin/Arzt 1992 Deutschland Anerkennung (15....,Deutschland (DE),Deutsch,Erteilt,"Zürich (2002), aktiv, MedBG, pr...",2002,Schweiz,NaN,2002,8620 Wetzikon,15583
5435,5435,0,medreg_93501.html,"Blasius, Elias John Achim",m,Kein Eintrag,2008,Deutschland,Anerkennung (22.06.2016),Ärztin/Arzt 2008 Deutschland Anerkennung (22....,Deutschland (DE),Deutsch,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2019,Deutschland,2019,2016,NaN,93501


In [181]:
df.drop(['Unnamed: 0'], axis=1)

,Unnamed: 0.1,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land,PP_Anerkennung,Anerkennung Arztdiplom,Ort,Doc_ID
0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Österreich (AT),Deutsch,Erteilt,"Aargau (2018), aktiv, MedBG, pr...",2013,Schweiz,NaN,2007,5620 Bremgarten AG,27259
1,0,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Basel-Land (1990), aktiv, MedBG...",1989,Schweiz,NaN,NaN,4132 Muttenz,55166
2,0,medreg_68248.html,"Klein, Katrin Stephanie",w,"Mühlemattstrasse 62,3007 Bern,BE",2000,Deutschland,Anerkennung (26.01.2011),Ärztin/Arzt 2000 Deutschland Anerkennung (26....,Deutschland (DE),Deutsch,Erteilt,"Bern (2011), aktiv, MedBG, priv...",2008,Deutschland,2016,2011,3007 Bern,68248
3,0,medreg_15778.html,"Leinfelder, Holger Ralf",m,"Austr. 79,4051 Basel,BS",1994,Deutschland,Anerkennung (23.07.2002),Ärztin/Arzt 1994 Deutschland Anerkennung (23....,Deutschland (DE),Deutsch,Erteilt,"Basel-Stadt (2007), aktiv, Unbe...",2002,Schweiz,NaN,2002,4051 Basel,15778
4,0,medreg_34926.html,"Jobin, Peter",m,"Standweg 20,2540 Grenchen,SO",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Solothurn (1990), aktiv, MedBG,...",1990,Schweiz,NaN,NaN,2540 Grenchen,34926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5432,0,medreg_17947.html,"Marti, Aurélie Sabine Daphné",w,"Rue des Lattes 59,1217 Meyrin,GE",2004,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 2004 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"Genf (2013), aktiv, MedBG, priv...",2013,Schweiz,NaN,NaN,1217 Meyrin,17947
5433,0,medreg_55667.html,"Nick, Beat",m,"Weissensteinstrasse ,4503 Solothurn,SO",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Solothurn (1998), aktiv, Kt. Re...",1992,Schweiz,NaN,NaN,4503 Solothurn,55667
5434,0,medreg_15583.html,"Jung, Gerhard Franz",m,"Leutholdstr. 4,8620 Wetzikon,ZH",1992,Deutschland,Anerkennung (15.07.2002),Ärztin/Arzt 1992 Deutschland Anerkennung (15....,Deutschland (DE),Deutsch,Erteilt,"Zürich (2002), aktiv, MedBG, pr...",2002,Schweiz,NaN,2002,8620 Wetzikon,15583
5435,0,medreg_93501.html,"Blasius, Elias John Achim",m,Kein Eintrag,2008,Deutschland,Anerkennung (22.06.2016),Ärztin/Arzt 2008 Deutschland Anerkennung (22....,Deutschland (DE),Deutsch,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2019,Deutschland,2019,2016,NaN,93501


In [182]:
df.drop(['Unnamed: 0.1'], axis=1)

,Unnamed: 0,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land,PP_Anerkennung,Anerkennung Arztdiplom,Ort,Doc_ID
0,0,medreg_27259.html,"Withalm, Gerhard",m,"Zürcherstrasse 4,5620 Bremgarten AG,AG",2006,Österreich,Anerkennung (28.03.2007),Ärztin/Arzt 2006 Österreich Anerkennung (28.0...,Österreich (AT),Deutsch,Erteilt,"Aargau (2018), aktiv, MedBG, pr...",2013,Schweiz,NaN,2007,5620 Bremgarten AG,27259
1,1,medreg_55166.html,"Schneider, Werner",m,"Hauptstr. 54,4132 Muttenz,BL",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Basel-Land (1990), aktiv, MedBG...",1989,Schweiz,NaN,NaN,4132 Muttenz,55166
2,2,medreg_68248.html,"Klein, Katrin Stephanie",w,"Mühlemattstrasse 62,3007 Bern,BE",2000,Deutschland,Anerkennung (26.01.2011),Ärztin/Arzt 2000 Deutschland Anerkennung (26....,Deutschland (DE),Deutsch,Erteilt,"Bern (2011), aktiv, MedBG, priv...",2008,Deutschland,2016,2011,3007 Bern,68248
3,3,medreg_15778.html,"Leinfelder, Holger Ralf",m,"Austr. 79,4051 Basel,BS",1994,Deutschland,Anerkennung (23.07.2002),Ärztin/Arzt 1994 Deutschland Anerkennung (23....,Deutschland (DE),Deutsch,Erteilt,"Basel-Stadt (2007), aktiv, Unbe...",2002,Schweiz,NaN,2002,4051 Basel,15778
4,4,medreg_34926.html,"Jobin, Peter",m,"Standweg 20,2540 Grenchen,SO",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Solothurn (1990), aktiv, MedBG,...",1990,Schweiz,NaN,NaN,2540 Grenchen,34926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5432,5432,medreg_17947.html,"Marti, Aurélie Sabine Daphné",w,"Rue des Lattes 59,1217 Meyrin,GE",2004,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 2004 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"Genf (2013), aktiv, MedBG, priv...",2013,Schweiz,NaN,NaN,1217 Meyrin,17947
5433,5433,medreg_55667.html,"Nick, Beat",m,"Weissensteinstrasse ,4503 Solothurn,SO",1982,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1982 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Solothurn (1998), aktiv, Kt. Re...",1992,Schweiz,NaN,NaN,4503 Solothurn,55667
5434,5434,medreg_15583.html,"Jung, Gerhard Franz",m,"Leutholdstr. 4,8620 Wetzikon,ZH",1992,Deutschland,Anerkennung (15.07.2002),Ärztin/Arzt 1992 Deutschland Anerkennung (15....,Deutschland (DE),Deutsch,Erteilt,"Zürich (2002), aktiv, MedBG, pr...",2002,Schweiz,NaN,2002,8620 Wetzikon,15583
5435,5435,medreg_93501.html,"Blasius, Elias John Achim",m,Kein Eintrag,2008,Deutschland,Anerkennung (22.06.2016),Ärztin/Arzt 2008 Deutschland Anerkennung (22....,Deutschland (DE),Deutsch,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2019,Deutschland,2019,2016,NaN,93501


In [183]:
df.to_csv(r'/Users/master/Desktop/CAS_DJ/MedReg/MedReg Scraping 2020/medreg_2020_psych_20200124_3.csv')

In [190]:
df2 = df.sort_values('Doc_ID')

In [192]:
df2 = df2.reset_index(drop=True)

In [195]:
df2.drop['Unnamed: 0', 'Unnamed: 0.1']

TypeError: 'method' object is not subscriptable

In [198]:
df2 = df2.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [199]:
df2

,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land,PP_Anerkennung,Anerkennung Arztdiplom,Ort,Doc_ID
0,medreg_552.html,"Poleski, Alexander",m,"Chemin des Allières 1,1012 Lausanne,VD",1985,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1985 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"Waadt (1986), aktiv, MedBG, pri...",1986,Schweiz,NaN,NaN,1012 Lausanne,552
1,medreg_566.html,"Langegger, Florian",m,"Seestrasse 22,8592 Uttwil,TG",1985,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1985 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Thurgau (2017), aktiv, MedBG, p...",1985,Schweiz,NaN,NaN,8592 Uttwil,566
2,medreg_590.html,"Rivera Prada, Jaime",m,Kein Eintrag,1985,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1985 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"Wallis (2014), inaktiv, Unbekannt",1987,Schweiz,NaN,NaN,NaN,590
3,medreg_611.html,"Dentoni, Roberto",m,"via Simen 10,6900 Lugano,TI",1986,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1986 Schweiz Eidgenössisches Dipl...,Schweiz (CH),"Französisch, Italienisch",Erteilt,"Tessin (1997), aktiv, MedBG, pr...",1991,Schweiz,NaN,NaN,6900 Lugano,611
4,medreg_617.html,"Anastopoulos, Anastasio",m,Kein Eintrag,1986,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1986 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"Waadt (2017), inaktiv, MedBG, p...",1986,Schweiz,NaN,NaN,NaN,617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5432,medreg_113646.html,"Mühlhauser, Markus Gerhard",m,Kein Eintrag,1999,Deutschland,Anerkennung (11.12.2019),Ärztin/Arzt 1999 Deutschland Anerkennung (11....,Deutschland (DE),,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2014,Deutschland,2019,2019,NaN,113646
5433,medreg_113811.html,"Noller, Silvia-Corina",w,Kein Eintrag,1985,Rumänien,Anerkennung (08.01.2020),Ärztin/Arzt 1985 Rumänien Anerkennung (08.01....,Deutschland (DE),"Deutsch, Französisch",Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2001,Deutschland,2020,2020,NaN,113811
5434,medreg_113880.html,"Fehr, Monika Hildegard",w,Kein Eintrag,1987,Deutschland,Anerkennung (08.01.2020),Ärztin/Arzt 1987 Deutschland Anerkennung (08....,Deutschland (DE),Deutsch,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,1995,Deutschland,2020,2020,NaN,113880
5435,medreg_113887.html,"Schneider, Hannah Adelheid",w,Kein Eintrag,2013,Deutschland,Anerkennung (08.01.2020),Ärztin/Arzt 2013 Deutschland Anerkennung (08....,Deutschland (DE),Deutsch,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2019,Deutschland,2020,2020,NaN,113887


In [200]:
df2.to_csv(r'/Users/master/Desktop/CAS_DJ/MedReg/MedReg Scraping 2020/medreg_2020_psych_20200124_4.csv')

In [202]:
df2.reset_index()

,index,Datei,Name,Geschlecht,Adresse,Jahr Diplom,Land Diplom,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB Kanton(e),PP_Jahr,PP_Land,PP_Anerkennung,Anerkennung Arztdiplom,Ort,Doc_ID
0,0,medreg_552.html,"Poleski, Alexander",m,"Chemin des Allières 1,1012 Lausanne,VD",1985,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1985 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"Waadt (1986), aktiv, MedBG, pri...",1986,Schweiz,NaN,NaN,1012 Lausanne,552
1,1,medreg_566.html,"Langegger, Florian",m,"Seestrasse 22,8592 Uttwil,TG",1985,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1985 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Thurgau (2017), aktiv, MedBG, p...",1985,Schweiz,NaN,NaN,8592 Uttwil,566
2,2,medreg_590.html,"Rivera Prada, Jaime",m,Kein Eintrag,1985,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1985 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"Wallis (2014), inaktiv, Unbekannt",1987,Schweiz,NaN,NaN,NaN,590
3,3,medreg_611.html,"Dentoni, Roberto",m,"via Simen 10,6900 Lugano,TI",1986,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1986 Schweiz Eidgenössisches Dipl...,Schweiz (CH),"Französisch, Italienisch",Erteilt,"Tessin (1997), aktiv, MedBG, pr...",1991,Schweiz,NaN,NaN,6900 Lugano,611
4,4,medreg_617.html,"Anastopoulos, Anastasio",m,Kein Eintrag,1986,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1986 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"Waadt (2017), inaktiv, MedBG, p...",1986,Schweiz,NaN,NaN,NaN,617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5432,5432,medreg_113646.html,"Mühlhauser, Markus Gerhard",m,Kein Eintrag,1999,Deutschland,Anerkennung (11.12.2019),Ärztin/Arzt 1999 Deutschland Anerkennung (11....,Deutschland (DE),,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2014,Deutschland,2019,2019,NaN,113646
5433,5433,medreg_113811.html,"Noller, Silvia-Corina",w,Kein Eintrag,1985,Rumänien,Anerkennung (08.01.2020),Ärztin/Arzt 1985 Rumänien Anerkennung (08.01....,Deutschland (DE),"Deutsch, Französisch",Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2001,Deutschland,2020,2020,NaN,113811
5434,5434,medreg_113880.html,"Fehr, Monika Hildegard",w,Kein Eintrag,1987,Deutschland,Anerkennung (08.01.2020),Ärztin/Arzt 1987 Deutschland Anerkennung (08....,Deutschland (DE),Deutsch,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,1995,Deutschland,2020,2020,NaN,113880
5435,5435,medreg_113887.html,"Schneider, Hannah Adelheid",w,Kein Eintrag,2013,Deutschland,Anerkennung (08.01.2020),Ärztin/Arzt 2013 Deutschland Anerkennung (08....,Deutschland (DE),Deutsch,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2019,Deutschland,2020,2020,NaN,113887


In [212]:
df = df.rename(columns = {'Anerkennung Arztdiplom':'Arzt_Anerkennung_Jahr', 'Jahr Diplom':'Arzt_Jahr', 'Land Diplom': 'Arzt_Land', 'BAB Kanton(e)':'BAB'})

In [213]:
df

,Datei,Name,Geschlecht,Adresse,Arzt_Jahr,Arzt_Land,Anerkennung,Ausbildung,Nationalität,Sprache,BAB Status,BAB,PP_Jahr,PP_Land,PP_Anerkennung,Arzt_Anerkennung_Jahr,Ort,Doc_ID
0,medreg_552.html,"Poleski, Alexander",m,"Chemin des Allières 1,1012 Lausanne,VD",1985,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1985 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"Waadt (1986), aktiv, MedBG, pri...",1986,Schweiz,NaN,NaN,1012 Lausanne,552
1,medreg_566.html,"Langegger, Florian",m,"Seestrasse 22,8592 Uttwil,TG",1985,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1985 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Deutsch,Erteilt,"Thurgau (2017), aktiv, MedBG, p...",1985,Schweiz,NaN,NaN,8592 Uttwil,566
2,medreg_590.html,"Rivera Prada, Jaime",m,Kein Eintrag,1985,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1985 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"Wallis (2014), inaktiv, Unbekannt",1987,Schweiz,NaN,NaN,NaN,590
3,medreg_611.html,"Dentoni, Roberto",m,"via Simen 10,6900 Lugano,TI",1986,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1986 Schweiz Eidgenössisches Dipl...,Schweiz (CH),"Französisch, Italienisch",Erteilt,"Tessin (1997), aktiv, MedBG, pr...",1991,Schweiz,NaN,NaN,6900 Lugano,611
4,medreg_617.html,"Anastopoulos, Anastasio",m,Kein Eintrag,1986,Schweiz,Eidgenössisches Diplom,Ärztin/Arzt 1986 Schweiz Eidgenössisches Dipl...,Schweiz (CH),Französisch,Erteilt,"Waadt (2017), inaktiv, MedBG, p...",1986,Schweiz,NaN,NaN,NaN,617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5432,medreg_113646.html,"Mühlhauser, Markus Gerhard",m,Kein Eintrag,1999,Deutschland,Anerkennung (11.12.2019),Ärztin/Arzt 1999 Deutschland Anerkennung (11....,Deutschland (DE),,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2014,Deutschland,2019,2019,NaN,113646
5433,medreg_113811.html,"Noller, Silvia-Corina",w,Kein Eintrag,1985,Rumänien,Anerkennung (08.01.2020),Ärztin/Arzt 1985 Rumänien Anerkennung (08.01....,Deutschland (DE),"Deutsch, Französisch",Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2001,Deutschland,2020,2020,NaN,113811
5434,medreg_113880.html,"Fehr, Monika Hildegard",w,Kein Eintrag,1987,Deutschland,Anerkennung (08.01.2020),Ärztin/Arzt 1987 Deutschland Anerkennung (08....,Deutschland (DE),Deutsch,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,1995,Deutschland,2020,2020,NaN,113880
5435,medreg_113887.html,"Schneider, Hannah Adelheid",w,Kein Eintrag,2013,Deutschland,Anerkennung (08.01.2020),Ärztin/Arzt 2013 Deutschland Anerkennung (08....,Deutschland (DE),Deutsch,Direktabgabe von Arzneimitteln gemäss kant. Be...,Kein Eintrag,2019,Deutschland,2020,2020,NaN,113887


In [227]:
df.columns

Index(['Datei', 'Name', 'Geschlecht', 'Adresse', 'Arzt_Jahr', 'Arzt_Land',
       'Anerkennung', 'Ausbildung', 'Nationalität', 'Sprache', 'BAB Status',
       'BAB', 'PP_Jahr', 'PP_Land', 'PP_Anerkennung', 'Arzt_Anerkennung_Jahr',
       'Ort', 'Doc_ID'],
      dtype='object')

In [228]:
df3 = df[['Doc_ID', 'Name', 'Geschlecht', 'Adresse', 'Ort', 'Nationalität', 'Sprache', 'Arzt_Jahr', 'Arzt_Land', 'Arzt_Anerkennung_Jahr', 
          'PP_Jahr', 'PP_Land', 'PP_Anerkennung', 'BAB', 'BAB Status', 'Datei']]

In [229]:
df3

,Doc_ID,Name,Geschlecht,Adresse,Ort,Nationalität,Sprache,Arzt_Jahr,Arzt_Land,Arzt_Anerkennung_Jahr,PP_Jahr,PP_Land,PP_Anerkennung,BAB,BAB Status,Datei
0,552,"Poleski, Alexander",m,"Chemin des Allières 1,1012 Lausanne,VD",1012 Lausanne,Schweiz (CH),Französisch,1985,Schweiz,NaN,1986,Schweiz,NaN,"Waadt (1986), aktiv, MedBG, pri...",Erteilt,medreg_552.html
1,566,"Langegger, Florian",m,"Seestrasse 22,8592 Uttwil,TG",8592 Uttwil,Schweiz (CH),Deutsch,1985,Schweiz,NaN,1985,Schweiz,NaN,"Thurgau (2017), aktiv, MedBG, p...",Erteilt,medreg_566.html
2,590,"Rivera Prada, Jaime",m,Kein Eintrag,NaN,Schweiz (CH),Französisch,1985,Schweiz,NaN,1987,Schweiz,NaN,"Wallis (2014), inaktiv, Unbekannt",Erteilt,medreg_590.html
3,611,"Dentoni, Roberto",m,"via Simen 10,6900 Lugano,TI",6900 Lugano,Schweiz (CH),"Französisch, Italienisch",1986,Schweiz,NaN,1991,Schweiz,NaN,"Tessin (1997), aktiv, MedBG, pr...",Erteilt,medreg_611.html
4,617,"Anastopoulos, Anastasio",m,Kein Eintrag,NaN,Schweiz (CH),Französisch,1986,Schweiz,NaN,1986,Schweiz,NaN,"Waadt (2017), inaktiv, MedBG, p...",Erteilt,medreg_617.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5432,113646,"Mühlhauser, Markus Gerhard",m,Kein Eintrag,NaN,Deutschland (DE),,1999,Deutschland,2019,2014,Deutschland,2019,Kein Eintrag,Direktabgabe von Arzneimitteln gemäss kant. Be...,medreg_113646.html
5433,113811,"Noller, Silvia-Corina",w,Kein Eintrag,NaN,Deutschland (DE),"Deutsch, Französisch",1985,Rumänien,2020,2001,Deutschland,2020,Kein Eintrag,Direktabgabe von Arzneimitteln gemäss kant. Be...,medreg_113811.html
5434,113880,"Fehr, Monika Hildegard",w,Kein Eintrag,NaN,Deutschland (DE),Deutsch,1987,Deutschland,2020,1995,Deutschland,2020,Kein Eintrag,Direktabgabe von Arzneimitteln gemäss kant. Be...,medreg_113880.html
5435,113887,"Schneider, Hannah Adelheid",w,Kein Eintrag,NaN,Deutschland (DE),Deutsch,2013,Deutschland,2020,2019,Deutschland,2020,Kein Eintrag,Direktabgabe von Arzneimitteln gemäss kant. Be...,medreg_113887.html


In [230]:
df3['BAB Status'].value_counts()

Erteilt                                                                           4428
Direktabgabe von Arzneimitteln gemäss kant. Bestimmungen (Selbstdispensation)      986
90-Tage Dienstleister                                                               12
Keine Bewilligung                                                                   11
Name: BAB Status, dtype: int64

In [233]:
df3['BAB Status'] = df3['BAB Status'].str.replace(r'Direktabgabe von Arzneimitteln .*', 'Kein Eintrag')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [234]:
df3['BAB Status'].value_counts()

Erteilt                  4428
Kein Eintrag              986
90-Tage Dienstleister      12
Keine Bewilligung          11
Name: BAB Status, dtype: int64

Ok. Nach dem mehrstufigen (und -stündigen) Prozess der Datenbereigung haben wir jetzt (hoffentlich) ein Dataframe bereit, das für die ersten geplanten Analysen bereitstehen sollte. Wir machen weiter mit dem [Notebook MedReg_Analyse_02](MedReg_Analyse_02.ipynb)

In [235]:
df3

,Doc_ID,Name,Geschlecht,Adresse,Ort,Nationalität,Sprache,Arzt_Jahr,Arzt_Land,Arzt_Anerkennung_Jahr,PP_Jahr,PP_Land,PP_Anerkennung,BAB,BAB Status,Datei
0,552,"Poleski, Alexander",m,"Chemin des Allières 1,1012 Lausanne,VD",1012 Lausanne,Schweiz (CH),Französisch,1985,Schweiz,NaN,1986,Schweiz,NaN,"Waadt (1986), aktiv, MedBG, pri...",Erteilt,medreg_552.html
1,566,"Langegger, Florian",m,"Seestrasse 22,8592 Uttwil,TG",8592 Uttwil,Schweiz (CH),Deutsch,1985,Schweiz,NaN,1985,Schweiz,NaN,"Thurgau (2017), aktiv, MedBG, p...",Erteilt,medreg_566.html
2,590,"Rivera Prada, Jaime",m,Kein Eintrag,NaN,Schweiz (CH),Französisch,1985,Schweiz,NaN,1987,Schweiz,NaN,"Wallis (2014), inaktiv, Unbekannt",Erteilt,medreg_590.html
3,611,"Dentoni, Roberto",m,"via Simen 10,6900 Lugano,TI",6900 Lugano,Schweiz (CH),"Französisch, Italienisch",1986,Schweiz,NaN,1991,Schweiz,NaN,"Tessin (1997), aktiv, MedBG, pr...",Erteilt,medreg_611.html
4,617,"Anastopoulos, Anastasio",m,Kein Eintrag,NaN,Schweiz (CH),Französisch,1986,Schweiz,NaN,1986,Schweiz,NaN,"Waadt (2017), inaktiv, MedBG, p...",Erteilt,medreg_617.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5432,113646,"Mühlhauser, Markus Gerhard",m,Kein Eintrag,NaN,Deutschland (DE),,1999,Deutschland,2019,2014,Deutschland,2019,Kein Eintrag,Kein Eintrag,medreg_113646.html
5433,113811,"Noller, Silvia-Corina",w,Kein Eintrag,NaN,Deutschland (DE),"Deutsch, Französisch",1985,Rumänien,2020,2001,Deutschland,2020,Kein Eintrag,Kein Eintrag,medreg_113811.html
5434,113880,"Fehr, Monika Hildegard",w,Kein Eintrag,NaN,Deutschland (DE),Deutsch,1987,Deutschland,2020,1995,Deutschland,2020,Kein Eintrag,Kein Eintrag,medreg_113880.html
5435,113887,"Schneider, Hannah Adelheid",w,Kein Eintrag,NaN,Deutschland (DE),Deutsch,2013,Deutschland,2020,2019,Deutschland,2020,Kein Eintrag,Kein Eintrag,medreg_113887.html


In [236]:
df3.to_csv(r'/Users/master/Desktop/CAS_DJ/MedReg/MedReg Scraping 2020/medreg_2020_psych_20200125.csv')